# Where you can find the code

* This notebook provides code for plotting of all the figures in "Reading minds"

* The code that ran the machine learning analysis that is plotted here can be found in run_2d_example_port2.py in the gitub repository.

* The core functions used to apply machine learning analysis to neuroimaging data can be found in SML_mvpa_scripts.py in the github repository.

In [12]:
import pandas as pd
import numpy as np
import matplotlib
import matplotlib.pyplot as plt 
import seaborn as sns
sns.set_style('ticks')

In [13]:
subject = 'am103'
data_dir = '/Users/sguerin/Dropbox/Stan_Server_Dropbox/portfolio_project/plotting_data'

# Fig. 1. 2D representation of decoding faces and scenes during encoding

In [17]:
# Load the data
twoD_enc_coef_df = pd.read_csv('{}/twoD_enc_coef_df_{}.csv'.format(data_dir, subject), index_col=0)
twoD_enc_df = pd.read_csv('{}/twoD_enc_df_{}.csv'.format(data_dir, subject), index_col=0)

twoD_enc_df.replace({'Picture Type': {'WF': 'Face', 'WP':'Place'}}, inplace=True)

# For rearranging we need to trial # to be unique by combining information with test run.
# Easiest way to do this is to code test run to the left of a decimal and trial # to the right of the decimanl
twoD_enc_df['Trial #'] = twoD_enc_df['Test Run'] + (twoD_enc_df['Trial #'] / 100)

# Rearrange for plotting with seaborn
pivot_df = pd.DataFrame.pivot(twoD_enc_df, index='Trial #', columns='Voxel Type', values='Mean Signal').reset_index()
reduced_df = twoD_enc_df.drop(['Voxel Type', 'Mean Signal'], axis=1).drop_duplicates()
twoD_enc_plot_df = pd.merge(pivot_df, reduced_df, how='inner', on='Trial #')


In [18]:
# Create a vector to represent the decison bound for the model in each cross-validation fold for plotting
# y = -(B1*x) / B2 when logit = 0 and thus p = .5
bound_df_list = list()

for run in range(5):
    B1 = twoD_enc_coef_df.loc[run, 'B1']
    B2 = twoD_enc_coef_df.loc[run, 'B2']

    x = np.linspace(-.5, .5, 25)
    y = (-1 * B1 * x) / B2
    
    bound_df_list.append(pd.DataFrame({'x' : x, 'y' : y}))

In [36]:
g = sns.lmplot(data=twoD_enc_plot_df, x='Face Voxels', y='Scene Voxels', hue='Picture Type', fit_reg=False,
               legend=False)
g.add_legend(label_order=['Place', 'Face'])
for run in range(5):
    plt.plot(bound_df_list[run].x, bound_df_list[run].y, color='black', alpha=.25)

plt.xlabel('Activity in Face Brain Regions')
plt.ylabel('Activity in Place Brain Regions')
plt.xlim(-1.5,1.5) # to help legend fit
plt.ylim(-1,1)
plt.savefig('Fig1.png', dpi=300, bbox_inches='tight')

# Fig. 2. 2D representation of decoding faces and scenes during recall

In [9]:
# Load the data
twoD_recall_coef_df = pd.read_csv('{}/twoD_recall_coef_df_{}.csv'.format(data_dir, subject), index_col=0)
twoD_recall_df = pd.read_csv('{}/twoD_recall_df_{}.csv'.format(data_dir, subject), index_col=0)

twoD_recall_df.replace({'Picture Type': {'WF': 'Face', 'WP':'Place'}}, inplace=True)

# Rearrange for plotting with seaborn
pivot_df = pd.DataFrame.pivot(twoD_recall_df, index='Trial #', columns='Voxel Type', values='Mean Signal').reset_index()
reduced_df = twoD_recall_df.drop(['Voxel Type', 'Mean Signal'], axis=1).drop_duplicates()
twoD_recall_plot_df = pd.merge(pivot_df, reduced_df, how='inner', on='Trial #')

In [10]:
# Create a vector to represent the decison bound for the model in each cross-validation fold for plotting
# y = -(B1*x) / B2 when logit = 0 and thus p = .5
twoD_recall_coef_df

bound_df_list = list()

run = 0

B1 = twoD_recall_coef_df.loc[run, 'B1']
B2 = twoD_recall_coef_df.loc[run, 'B2']

x = np.linspace(-.5, .5, 25)
y = (-1 * B1 * x) / B2

bound_df_list.append(pd.DataFrame({'x' : x, 'y' : y}))

In [35]:
g = sns.lmplot(data=twoD_recall_plot_df, x='Face Voxels', y='Scene Voxels', hue='Picture Type', fit_reg=False, 
               legend=False)
g.add_legend(label_order=['Place', 'Face'])

run = 0
plt.plot(bound_df_list[run].x, bound_df_list[run].y, color='black', alpha=.25)

plt.xlabel('Face Brain Regions')
plt.ylabel('Place Brain Regions')
plt.xlim(-1.5,1.5) # to help legend fit
plt.ylim(-1,1)
plt.savefig('Fig2.png', dpi=300, bbox_inches='tight')

# Figure 3: Correlating memory performance with classifier performance

In [70]:
# Import data from from the 1000D model
grandD_recall_auc_df = pd.read_csv('{}/grandD_recall_df.csv'.format(data_dir), index_col=0)
grandD_recall_auc_df = grandD_recall_auc_df.rename(columns={'AUC':'Model Performance'})

# Import behavioral data
temp_df = pd.read_csv('{}/usable_metrics_and_beh_basics.csv'.format(data_dir), index_col=0)
beh_df = temp_df.loc[:, ['am_num', 'age', 'dprime_all']]
beh_df.rename(columns={'dprime_all':'Memory Performance'}, inplace=True)
merged_df = grandD_recall_auc_df.merge(beh_df, left_on='Subject', right_on='am_num', how='inner')

g = sns.lmplot(data=merged_df, x='Model Performance', y='Memory Performance', fit_reg=True)
plt.savefig('Fig3.png', dpi=300, bbox_inches='tight')

# Figure 4: Correlating age with memory performance

In [87]:
g = sns.lmplot(data=merged_df, x='age', y='Memory Performance', fit_reg=True)
plt.xlabel('Age')
plt.savefig('Fig4.png', dpi=300, bbox_inches='tight')

# Figure 5:  Correlating memory performance with classifier performance after removing all variance attributable to age.

In [85]:
g = sns.lmplot(data=merged_df, x='Model Performance', y='Memory Performance', fit_reg=True,
              x_partial='age', y_partial='age')
plt.savefig('Fig5.png', dpi=300, bbox_inches='tight')